In [ ]:
__author__ = "Jose David Marroquin Toledo"
__credits__ = ["Jose David Marroquin Toledo",]
__email__ = "jose@marroquin.cl"
__status__ = "Development"

## 1. Forward Imaging Model
In this process, the sample is illuminated with one LED of the grid at time and a camera captures Lo-Res images under different incident angles. To simulate it, we have to:

1. Create a Hi-Res complex image.

2. Generate the incident wave vectors.

3. Produce the ouput Lo-Res images.

###  [2. The Recovery Process](phaseretrieval.ipynb)

In [ ]:
from PIL import Image
import numpy as np
import math
import matplotlib.pyplot as plt
import decimal
import scipy.misc
import os

In [ ]:
%matplotlib inline

In [ ]:
# 1. Create the Hi-Res complex image.
# Return the Hi-Res complex object ('numpy.ndarray'), the sample.
def generate_obj(amplitude, phase, **kwargs):
    # Show the output image in the notebook.
    show = kwargs.pop('show', False)
    amplitude = Image.open(amplitude)
    phase = Image.open(phase)
    h, w = amplitude.size
    # In MATLAB, imresize uses bicubic interpolation by default.
    phase = phase.resize((w, h), resample=Image.BICUBIC)
    # 'd' ('str') is a character code for a double-precision
    # floating-point number.
    arr_amplitude = np.array(amplitude, dtype='d')
    arr_phase = np.array(phase, dtype='d')
    amplitude.close()
    phase.close()
    arr_phase = math.pi * arr_phase / np.amax(arr_phase)
    obj = arr_amplitude * np.exp(1j * arr_phase)
    obj = np.absolute(obj)  # The Hi-Res complex image.
    if show:
        # Using PIL, the objects is shown as an image almost
        # completely white.
        # img_obj = Image.fromarray(obj)
        # img_obj.show()
        # Using matplotlib anc 'Greys_r' ('str') as value for 'cmap'
        # key [1], the object is shown like in Matlab R2009b.
        #
        # [1] unutbu. (2010). Display image as grayscale using matplotlib [Msg 1]. Message posted to
        # http://stackoverflow.com/questions/3823752/display-image-as-grayscale-using-matplotlib?answertab=votes#tab-top
        plt_img = plt.imshow(obj, cmap='Greys_r')
    return obj, w, h

In [ ]:
# 2. Generate the incident wave vectors.
# Return a leds_prc-by-leds_prc array ('numpy.ndarray') with the
# magnitude of the x-component, kx, and y-component, ky, of the wave
# vectors of the incident waves that emerge from the LED grid.
#
# led_dist ('int') and height ('int') are distances between LEDs and
# the grid and the sample, respectively. leds_prc by "LEDs per row and
# column".
def generate_wave_vectors(leds_prc, led_dist, height):
    x_max = math.floor((leds_prc / 2)) * led_dist
    x_min = -x_max
    y_max = x_max
    y_min = x_min
    l_row = list()
    l_arr = list()
    # xy_max (int) + 1 to include xy_max.
    l_range = list(range(x_min, x_max + 1, led_dist))
    for i in l_range:
        for j in l_range:
            l_row.append((j, -i))
        l_arr.append(l_row)
        l_row = list()
    arr = np.array(l_arr)  # arr ('numpy.ndarray') contains (x, y)
                           # coordinates.
    arr = arr / height
    arr = np.arctan(arr)
    arr = np.sin(arr)
    arr = -arr
    return arr

In [ ]:
# In Python 3, the round() function had changed. For example,
# round(2.5) returns 2 ('int') like round(1.5). It is possible to
# obtain 3 ('int') for 2.5 rounded using the decimal module [2].
#
# [2] Barthelemy. (2014). Python 3.x rounding behavior. Message posted to
# http://stackoverflow.com/questions/10825926/python-3-x-rounding-behavior
def round_half_up(num):
    return int(decimal.Decimal(num).quantize(decimal.Decimal(1),
                                             rounding=decimal.ROUND_HALF_UP))

In [ ]:
# Return the number of rows and columns of the Hi-Res output, vector
# waves, dkx ('float') and dky ('float'), and the coherent transfer
# function of the objective.
#
# wave_vectors is a numpy.ndarray, wavelen is a int, ccdpx ('float')
# is the sampling pixel size of the CCD, hirespx ('float') is the
# pixel size of the reconstruction, na ('float') is the numerical
# aperture of the lens, w ('int') the width in pixels of the output
# Hi-Res image and h ('int') its height. zoom ('int') is a STRANGE
# PARAMETER but it acts as it were a zoom.
def get_cft(wave_vectors, wavelen, ccdpx, zoom, na, w, h):
    k_0 = 2 * math.pi / wavelen;
    hirespx = ccdpx / zoom
    p = int(h / (ccdpx / hirespx))  # Number of rows of the output.
    q = int(w / (ccdpx / hirespx))  # Number of columns of the output.
    img_seq_lores = list()
    k = k_0 * wave_vectors;
    # From all rows, all columns, extract the first element.
    kx = k[:, :, 0]
    # Reshape xk ('numpy.ndarray') in 1-D array.
    kx = np.reshape(kx, len(kx.flat))
    ky = k[:, :, 1]
    ky = np.reshape(ky, len(ky.flat))
    dkx = 2 * math.pi / (hirespx * w)
    dky = 2 * math.pi / (hirespx * h)
    cutoff_freq = na * k_0
    k_max = math.pi / ccdpx
    kxm, kym = np.meshgrid(np.arange(-k_max, k_max + 1,
                                     k_max / ((q - 1) / 2)),
                           np.arange(-k_max, k_max + 1,
                                     k_max / ((q - 1) / 2)))
    coherent_transfer_funct = ((kxm ** 2 + kym ** 2) < cutoff_freq ** 2)
    # E.g., convert from [[False, True, ..., False, False],
    #                     ...,
    #                     [True, False, ..., True, True]]
    #                 to [[0, 1, ..., 0, 0],
    #                     , ...,
    #                     [1, 0, ..., 1, 1]] ('numpy.ndarray').
    coherent_transfer_funct = coherent_transfer_funct.astype(float)
    return p, q, kx, ky, dkx, dky, coherent_transfer_funct

In [ ]:
# Return an array ('numpy.ndarray') with the sequence of Lo-Res
# images which are also stored. Each image only contains the
# amplitude information.
def generate_lores_set(obj, cft, leds_prc, kx, ky, m, n, dkx, dky, p,
                       q, **kwargs):
    outpath = kwargs.pop('outpath', 'img-lores/')
    prefix = kwargs.pop('prefix', 'lores_')
    extension = kwargs.pop('extension', '.tiff')
    obj_ft = np.fft.fftshift(np.fft.fft2(obj))
    img_seq_lores = list()
    for i in range(leds_prc ** 2):
        kxc = round_half_up((m + 1) / 2.0 + kx[i] / dkx)
        kyc = round_half_up((m + 1) / 2.0 + ky[i] / dky)
        kyl = round_half_up(kyc - (p - 1) / 2.0);
        kyh = round_half_up(kyc + (p - 1) / 2.0);
        kxl = round_half_up(kxc - (q - 1) / 2.0);
        kxh = round_half_up(kxc + (q - 1) / 2.0);
        img_seq_lores_ft = (p / m) ** 2
        img_seq_lores_ft *= obj_ft[kyl - 1:kyh, kxl - 1:kxh]
        img_seq_lores_ft *= cft
        img_lores = np.absolute(np.fft.ifft2(np.fft.ifftshift(img_seq_lores_ft)))
        img_seq_lores.append(img_lores)
        # scipy.misc.save() saves the TIFF file as 'uint8' [3].
        #
        # [3] Olsson, T. (2015). Saving 16-bit tiff files using Python. Retrieved from
        # http://tjelvarolsson.com/blog/saving-16bit-tiff-files-using-python/
        if not os.path.exists(outpath):
            os.makedirs(outpath)
        scipy.misc.imsave(outpath + prefix + str(i) + extension, img_lores)
    # plt_img = plt.imshow(img_seq_lores[109], cmap='Greys_r')
    return np.array(img_seq_lores)    

In [ ]:
# Return a string that contains a sequence n-zeros followed by
# num ('int') as 'str', for example, num_str_zeros(89, 4) returns
# '0089'.
def num_str_zeros(num, ref):
    len_num = len(str(num))
    str_num = ''
    for i in range(len(ref) - len_num):
        str_num += '0'
    str_num += str(num)
    return str_num

In [ ]:
# Return a list with the image indexes to use. Using shift ('int'), it
# is possible to change the central image.
def get_idx_img_to_use(lenrow, lenuse, **kwargs):
    matlab = kwargs.pop('matlab', False)
    # To approximate even numbers.
    lower = kwargs.pop('lower', False)
    # Begin the selection down ('int') rows down.
    down = kwargs.pop('down', 0)
    # Begin the selection right ('int') columns right.
    right = kwargs.pop('right', 0)
    shift = kwargs.pop('shift', 0)
    # The even numbers are approximated to the nearest lower integer.
    if lower:
        if lenrow % 2 == 0:
            lenrow -= 1
        if lenuse % 2 == 0:
            lenuse -= 1
    else:
        if lenrow % 2 == 0:
            lenrow += 1
        if lenuse % 2 == 0:
            lenuse += 1
    arr = np.arange(lenrow ** 2)
    if matlab:  # In Matlab, the indexes begin in 1.
        arr = np.arange(1, lenrow ** 2 + 1)
    arr = np.split(arr, lenrow)  # Here, arr is type 'list'.
    # Convert arr ('list') to 'numpy.ndarray' to use advanced
    # indexing, for example, arr[0:2, 9:89].
    arr = np.array(arr)
    limit_l_up = int((lenrow - lenuse) / 2)
    limit_r_down = lenrow - limit_l_up
    if shift != 0:  # Roll arr ('numpy.ndarray'). A negative
                    # rotate it to left and positive number, to
                    # right.
        arr = np.roll(arr, shift)
    arr = arr[limit_l_up - down :limit_r_down - down, limit_l_up + right:limit_r_down + right]
    return list(arr.flat)

In [ ]:
# Return a array with the Lo-Res images as arrays.
def get_set_from_folder(**kwargs):
    path = kwargs.pop('path', 'img-lores/')
    prefix = kwargs.pop('prefix', 'lores_')
    first = kwargs.pop('first', '001')
    extension = kwargs.pop('extension', '.tif')
    # Sub selection of the set. subsel ('numpy.ndarray') contains
    # the indexes of the images.
    subsel = kwargs.pop('subsel', list())
    n = kwargs.pop('n', 293)
    l_imgs = list()
    i = int(first)
    path = path + prefix
    for i in range(len(subsel)):
        filepath = path + num_str_zeros(subsel[i], first) + extension 
        print(filepath)
        try:
            img = Image.open(filepath)
            arr_img = np.array(img, dtype='d')
            img.close()
            l_imgs.append(arr_img)
        except IOError:
            print('No such file or directory: ' + filepath)
    return np.array(l_imgs)